In [86]:
import torch
import torch.optim as optim
import torch.nn as nn
from functions import base_functions
from tqdm import tqdm

In [87]:
from numpy import double

class SymbolicLayer(nn.Module):
    def __init__(self, input_size, output_size, functions: list):
        super().__init__()
        self.functions = [func for func in functions]
        self.n_funcs = len(self.functions)
        self.output = None
        self.input_size = input_size

        self.single_input = []
        self.double_input = []

        # Check how many inputs each function have
        for f in functions:
            if f.n_inputs == 1: 
                self.single_input.append(f)
            if f.n_inputs == 2:
                self.double_input.append(f)
        
        self.n_single = len(self.single_input)
        self.n_double = len(self.double_input)
        
        # Adjust number of layers according to function inputs
        self.output_size = output_size
        self.W = torch.nn.Parameter(data=torch.ones((self.input_size, self.n_single + self.n_double * 2), requires_grad=True))

    def __call__(self, x):

        self.output = torch.tensor([])
        
        g = self.W.mul(x)

        out_i = in_i = 0

        while out_i  < self.n_single:
            self.output = torch.cat((self.output, (self.single_input[out_i].torch(g[:,in_i]))))
            out_i += 1
            in_i +=1

        while out_i < self.n_funcs:
            self.output = torch.cat((self.output, self.functions[out_i].torch(g[:,in_i], g[:,in_i + 1])))
            out_i += 1
            in_i += 2

        self.output = torch.stack([torch.tensor([torch.sum(self.output)]) for out in range(self.output_size)], )
        return self.output

In [88]:
class SymbolicNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, functions: list):
        super(SymbolicNet, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.functions = functions
        self.n_funcs = len(functions)
        self.output_W = torch.nn.Parameter(data=torch.ones(self.n_funcs, self.output_size))
        self.fc1 = SymbolicLayer(input_size, hidden_size, functions)
        self.fc2 = SymbolicLayer(hidden_size, output_size, functions)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = torch.tensor([x])
        return x
    
    def train(self, x, y, epochs=100, lr=0.01):
        optimizer = optim.SGD(self.parameters(), lr=lr)
        loss_fn = nn.MSELoss()
        for epoch in range(epochs):
            total_loss = 0
            for i in range(len(x)):
                optimizer.zero_grad()
                output = self.forward(x[i])
                loss = loss_fn(output, y[i])
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
        
            if epoch % 10 == 0:
                print(f'Epoch: {epoch} Loss: {loss.item()}')

In [89]:
model = SymbolicNet(1, 10, 1, base_functions)

In [85]:

x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=torch.float32, requires_grad=True)
y = torch.tensor(torch.square(x), requires_grad=True)

model.train(x, y)

/tmp/ipykernel_31112/4191838912.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(torch.square(x), requires_grad=True)


Epoch: 0 Loss: 9762646016.0
Epoch: 10 Loss: 9762646016.0
Epoch: 20 Loss: 9762646016.0
Epoch: 30 Loss: 9762646016.0
Epoch: 40 Loss: 9762646016.0
Epoch: 50 Loss: 9762646016.0
Epoch: 60 Loss: 9762646016.0
Epoch: 70 Loss: 9762646016.0
Epoch: 80 Loss: 9762646016.0
Epoch: 90 Loss: 9762646016.0
